In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_full = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_full = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
sample_sumbission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

train_X = train_full.drop(columns = ['Transported','Name','Cabin'])
train_Y = train_full.loc[:,['Transported','PassengerId']]

sample_sumbission.head()


,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [3]:
train_X.HomePlanet.fillna('Earth', inplace = True) #Earth is the most common Home Planet 
train_X.CryoSleep.fillna(False, inplace = True) #if its Null then probably they aren,t CryoSleeping i think it would have been documented
train_X.Destination.fillna('TRAPPIST-1e', inplace = True) #the most common destination
train_X.Age.fillna(29, inplace = True) #mean value of all ages
train_X.VIP.fillna(False, inplace = True) #nah they would be listed as vip probably if they were
#all these nulls are replaced with 0 because 97% of passengers haven,t paid for luxury goods
train_X.RoomService.fillna(0, inplace = True)
train_X.FoodCourt.fillna(0, inplace = True)
train_X.ShoppingMall.fillna(0, inplace = True)
train_X.Spa.fillna(0, inplace = True)
train_X.VRDeck.fillna(0, inplace = True)
train_X.set_index('PassengerId',inplace = True)
train_Y.set_index('PassengerId',inplace = True)
print(train_X.isnull().sum())

print(round(train_X.Age.mean().sum()))
print(train_X.dtypes)
train_X.head()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64
29
HomePlanet       object
CryoSleep          bool
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
PassengerId,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0


In [4]:
test_full.HomePlanet.fillna('Earth', inplace = True) #Earth is the most common Home Planet 
test_full.CryoSleep.fillna(False, inplace = True) #if its Null then probably they aren,t CryoSleeping i think it would have been documented
test_full.Destination.fillna('TRAPPIST-1e', inplace = True) #the most common destination
test_full.Age.fillna(29, inplace = True) #mean value of all ages
test_full.VIP.fillna(False, inplace = True) #nah they would be listed as vip probably if they were
#all these nulls are replaced with 0 because 97% of passengers haven,t paid for luxury goods
test_full.RoomService.fillna(0, inplace = True)
test_full.FoodCourt.fillna(0, inplace = True)
test_full.ShoppingMall.fillna(0, inplace = True)
test_full.Spa.fillna(0, inplace = True)
test_full.VRDeck.fillna(0, inplace = True)

test_full.set_index('PassengerId',inplace = True)
print(train_X.isnull().sum())

test_full.head()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
#Better solution for encoding categorical data than pd.get_dummies
import sklearn.preprocessing
encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown = 'ignore')

from sklearn.pipeline import Pipeline
categorical_processing = Pipeline(steps=[('encoder', encoder)])
from sklearn.compose import ColumnTransformer

# create the ColumnTransormer object
All_categorical_preprocessing = ColumnTransformer(transformers=[('categorical', categorical_processing, ['HomePlanet', 'CryoSleep','Destination','VIP'])],remainder='passthrough')

In [6]:
from sklearn.model_selection import train_test_split
train_Y.Transported.replace([False, True],
                        [0, 1], inplace=True)
train_Y.head()
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, random_state=0)


In [7]:
print(train_X.shape)
print(val_X.shape)
print(train_Y.shape)
print(val_Y.shape)

(6519, 10)
(2174, 10)
(6519, 1)
(2174, 1)


In [8]:

train_X = All_categorical_preprocessing.fit_transform(train_X)
test_data =  All_categorical_preprocessing.transform(test_full)
val_X =  All_categorical_preprocessing.fit_transform(val_X)


print(f'Number of features in training set: {train_X.shape}')
print(f'Number of features in testing set: {test_data.shape}')
features = train_X.shape[1]

Number of features in training set: (6519, 16)
Number of features in testing set: (4277, 16)


In [9]:
print(train_X[0])

[ 0.  0.  1.  0.  1.  0.  0.  1.  1.  0. 37.  0.  0.  0.  0.  0.]


In [10]:
import tensorflow as tf
#normalizer = tf.keras.layers.Normalization(axis = -1)
#normalizer.adapt(train_X)
model = tf.keras.models.Sequential([
    #normalizer,
    tf.keras.layers.Dense(512,activation = 'relu', input_shape = [16]),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam' , loss = 'BinaryCrossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 140,289
Trainable params: 140,289
Non-trainable params: 0
_________________________________________________________________


2022-12-06 20:09:25.302835: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

In [12]:
model.fit(train_X, train_Y, validation_split = 0.2,  epochs= 35, callbacks=[early_stopping])

2022-12-06 20:09:25.521467: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/35
163/163 [==============================] - 1s 5ms/step - loss: 9.8741 - accuracy: 0.7561 - val_loss: 4.5331 - val_accuracy: 0.7914
Epoch 2/35
163/163 [==============================] - 1s 4ms/step - loss: 2.5691 - accuracy: 0.7576 - val_loss: 1.9961 - val_accuracy: 0.7883
Epoch 3/35
163/163 [==============================] - 1s 4ms/step - loss: 1.8748 - accuracy: 0.7651 - val_loss: 1.5622 - val_accuracy: 0.7791
Epoch 4/35
163/163 [==============================] - 1s 4ms/step - loss: 1.4698 - accuracy: 0.7597 - val_loss: 1.4304 - val_accuracy: 0.7768
Epoch 5/35
163/163 [==============================] - 1s 4ms/step - loss: 0.9119 - accuracy: 0.7653 - val_loss: 1.3062 - val_accuracy: 0.7830
Epoch 6/35
163/163 [==============================] - 1s 4ms/step - loss: 0.7542 - accuracy: 0.7733 - val_loss: 0.6508 - val_accuracy: 0.7914
Epoch 7/35
163/163 [==============================] - 1s 4ms/step - loss: 0.6264 - accuracy: 0.7768 - val_loss: 0.5049 - val_accuracy: 0.7776
Epoch 

In [13]:
aa = model.predict(test_data)
model.evaluate(val_X, val_Y)

68/68 [==============================] - 0s 2ms/step - loss: 0.4502 - accuracy: 0.7962


[0.4501548707485199, 0.7962281703948975]

In [14]:
print(aa[0])

[0.6006521]


In [15]:
test_full['Transported'] = aa
test_full.reset_index(inplace = True)
test_full.Transported  = test_full.Transported.map(lambda b: True if (b > 0.5) else False )
score = test_full.loc[:,['PassengerId','Transported']]
score.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [16]:
output = score.to_csv('predictions2.csv', index = False)